In [67]:
import os
import time

In [68]:
os.path.getsize('C:\\Users\\Partners\\AppData\\Local\\Temp\\Temp1_archive.zip\\Hotel Reservations.csv')

3240604

In [69]:
#Reading file with pandas

In [70]:
import pandas as pd

start = time.time()
pd_data = pd.read_csv('C:\\Users\\Partners\\AppData\\Local\\Temp\\Temp1_archive.zip\\Hotel Reservations.csv')
end = time.time()

print("Read data with Pandas: ",(end-start),"sec")

Read data with Pandas:  0.2634611129760742 sec


In [71]:
#Reading with Dask
from dask import dataframe as dd

start = time.time()
dask_data = dd.read_csv('C:\\Users\\Partners\\AppData\\Local\\Temp\\Temp1_archive.zip\\Hotel Reservations.csv')
end = time.time()

print("Read data with Dask: ",(end-start),"sec")

Read data with Dask:  0.023997068405151367 sec


In [72]:
#It can be concluded that Dask is faster compared to pandas 

In [73]:
from dask import dataframe as dd
d_data= dd.read_csv('C:\\Users\\Partners\\AppData\\Local\\Temp\\Temp1_archive.zip\\Hotel Reservations.csv')

In [74]:
d_data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 19 entries, Booking_ID to booking_status
dtypes: object(5), float64(1), int64(13)

In [75]:
#Removing underscores
d_data.columns=d_data.columns.str.replace('[_,@,&]','')

C:\Users\Partners\AppData\Local\Temp\ipykernel_2288\2720405805.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  d_data.columns=d_data.columns.str.replace('[_,@,&]','')


In [76]:
#To remove white space from columns
d_data.columns = d_data.columns.str.replace(' ', '')

In [77]:
d_data.columns

Index(['BookingID', 'noofadults', 'noofchildren', 'noofweekendnights',
       'noofweeknights', 'typeofmealplan', 'requiredcarparkingspace',
       'roomtypereserved', 'leadtime', 'arrivalyear', 'arrivalmonth',
       'arrivaldate', 'marketsegmenttype', 'repeatedguest',
       'noofpreviouscancellations', 'noofpreviousbookingsnotcanceled',
       'avgpriceperroom', 'noofspecialrequests', 'bookingstatus'],
      dtype='object')

In [85]:


%%writefile Testutility.py

import yaml
import logging
import subprocess
import yaml
import datetime
import gc
import re 
import pandas as pd
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string


def col_header_val(df, table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Overwriting Testutility.py


In [86]:
%%writefile books.yaml
file_type: csv
dataset_name: file
file_name: books_rating
table_name : edsurv
inbound_delimiter : ","
outbound_delimiter : "|"
skip_leading_rows: 1
columns: 
    - Id
    - Title
    - Price
    - Userid
    - profileName
    - review/helpfulness
    - review/score
    - review/time
    - review/summary
    - review/text

Overwriting books.yaml


In [88]:
import Testutility as util

data_config =util.read_config_file("books.yaml")

In [89]:
data_config

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'books_rating',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Id',
  'Title',
  'Price',
  'Userid',
  'profileName',
  'review/helpfulness',
  'review/score',
  'review/time',
  'review/summary',
  'review/text']}

In [90]:
from dask import dataframe as dd
df_data= dd.read_csv('C:\\Users\\Partners\\AppData\\Local\\Temp\\Temp1_archive.zip\\Hotel Reservations.csv',dtype={'Id': 'object'})
df_data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [110]:
#Reading the file using config file
#Validating columns
file_type = data_config['file_type']
source_file = "C:\\Users\\Partners\\AppData\\Local\\Temp\\Temp1_archive.zip\\Hotel Reservations"  + f'.{file_type}'
df = pd.read_csv(source_file,data_config['inbound_delimiter'],encoding='ISO-8859-1')
df.head()

C:\Users\Partners\AppData\Local\Temp\ipykernel_2288\3171370150.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,data_config['inbound_delimiter'],encoding='ISO-8859-1')


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [114]:
util.col_header_val(df,data_config)

column name and column length validation failed
Following File columns are not in the YAML file ['no_of_adults', 'no_of_special_requests', 'market_segment_type', 'arrival_date', 'required_car_parking_space', 'room_type_reserved', 'type_of_meal_plan', 'no_of_previous_cancellations', 'booking_id', 'lead_time', 'booking_status', 'repeated_guest', 'arrival_month', 'no_of_children', 'no_of_week_nights', 'avg_price_per_room', 'arrival_year', 'no_of_weekend_nights', 'no_of_previous_bookings_not_canceled']
Following YAML columns are not in the file uploaded ['review/helpfulness', 'review/score', 'title', 'userid', 'review/time', 'id', 'price', 'profilename', 'review/text', 'review/summary']


0

In [115]:
print("columns of files are:" ,d_data.columns)
print("columns of YAML are:" ,data_config['columns'])

columns of files are: Index(['bookingid', 'noofadults', 'noofchildren', 'noofweekendnights',
       'noofweeknights', 'typeofmealplan', 'requiredcarparkingspace',
       'roomtypereserved', 'leadtime', 'arrivalyear', 'arrivalmonth',
       'arrivaldate', 'marketsegmenttype', 'repeatedguest',
       'noofpreviouscancellations', 'noofpreviousbookingsnotcanceled',
       'avgpriceperroom', 'noofspecialrequests', 'bookingstatus'],
      dtype='object')
columns of YAML are: ['Id', 'Title', 'Price', 'Userid', 'profileName', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']


In [122]:
filename = "Hotel Reservations/" + data_config['file_name']+ " out" + f'.{file_type}.gz'

df.to_csv(filename, sep=data_config['outbound_delimiter'], compression='gzip')


['C:\\Users\\Partners\\Hotel Reservations\\books_rating out.csv.gz\\0.part']

In [124]:
import os
origin_size = os.path.getsize(source_file)
compressed_size = os.path.getsize(filename)
number_rows = df.shape[0]
number_columns= df.shape[1]

In [125]:
print("Original size: " + f'{origin_size}' +"Ko")
print("Compressed size: " + f'{compressed_size}' +"Ko")
print("Number of rows: " + f'{number_rows}')
print("Number of columns: "+  f'{number_columns}')

Original size: 3240604Ko
Compressed size: 0Ko
Number of rows: Delayed('int-14e73334-591c-49ee-93ca-3c094841dd36')
Number of columns: 19
